In [1]:
from collections import deque, defaultdict, Counter
from heapq import heapify, heappush, heappop
import numpy as np
from copy import deepcopy
import math
import time
from functools import cache, reduce, cmp_to_key
import graphviz
from itertools import product
import matplotlib.pyplot as plt
from bisect import bisect_left, bisect_right
import json

In [2]:
valves = set()
flow = {}
adj = defaultdict(set)

with open("./data/day16.txt") as f:
    while line := f.readline():
        line = line.rstrip().replace(',', '').split(' ')
        valve = line[1]
        valves.add(valve)
        flow[valve] = int(line[4].replace(';','').split('=')[1])
        for next_valve in line[9:]:
            valves.add(next_valve)
            adj[valve].add(next_valve)
valves = sorted(list(valves))
valve_to_ind = {valve:i for i, valve in enumerate(valves)}
len(valves), valves[0], flow[valves[0]], adj[valves[0]]

(55, 'AA', 0, {'DI', 'NB', 'UV', 'VS', 'XO'})

# Part 1

First idea: DFS with keeping track of the state of valves to allow coming back to a valve if the state has changed, i.e. a new valve has been opened. This is just super slow though. Works, but prohibitively slow on real input.

In [3]:
def dfs(ind, state, visited, time_left):
    if time_left <= 1:
        return 0
    res = 0
    valve_open = (state & (1 << ind)) != 0
    if not valve_open and flow[valves[ind]] != 0:
        new_state = state | (1 << ind)
        res = (time_left-1)*flow[valves[ind]] + dfs(ind, new_state, {(ind, new_state)}, time_left-1)
    for a in adj[valves[ind]]:
        a_ind = valve_to_ind[a]
        if (a_ind, state) not in visited:
            visited.add((a_ind, state))
            res = max(res, dfs(a_ind, state, visited, time_left-1))
            visited.remove((a_ind, state))
    return res

Second idea: dynamic programming (implemented top-down, i.e. recursion with memoization).

In [4]:
@cache
def dp(ind, state, time_left):
    if time_left <= 1:
        return 0
    res = 0
    valve_open = (state & (1 << ind)) != 0
    if not valve_open and flow[valves[ind]] != 0:
        new_state = state | (1 << ind)
        res = (time_left-1)*flow[valves[ind]] + dp(ind, new_state, time_left-1)
    for a in adj[valves[ind]]:
        a_ind = valve_to_ind[a]
        res = max(res, dp(a_ind, state, time_left-1))
    return res

In [5]:
dp(0, 0, 30)

1617